In [1]:
import os 
import sys
import matplotlib.pyplot as plt 
import numpy as np 

import torch
import torch.nn as nn 
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

sys.path.append(r"..\Dataset\Segmentation")
sys.path.append(r"..\Backbone\Dense")

from Tiramisu import *
from VOCseg import *


In [2]:
model = Tiramisu103( num_classes=12, init_weight = False,)
#model.cuda()
model = nn.DataParallel(model)
model_chẹkpoints = torch.load(r'..\Models\Tiramisu\tiramisu103.pth', map_location="cuda")
print('loading Model')
model.load_state_dict(model_chẹkpoints)

loading Model


<All keys matched successfully>

In [3]:
model_one = Tiramisu103(num_classes=2, init_weight=True)
model_one = nn.DataParallel(model_one)


In [ ]:
# ==> for class 
def load_pretrained_weight(self, path_pretrained = r'..\Models\Tiramisu\tiramisu103.pth', watching = True):
    # load pretrained state dict
    pretrained_dict = torch.load(pretrained)
    # get state dict
    model_dict = self.state_dict()
    # 1. filter out unnecessary keys
    tmp_pretrained_dict = {}
    for k, v in list(pretrained_dict.items())[:-2]:
      if k in model_dict:
        if watching:
            print(f'match {k}')
        tmp_pretrained_dict.update({k: v})
      else:
        if watching:
            print(f'unmatch {k}')
    # 2. overwrite entries in the existing state dict
    model_dict.update(tmp_pretrained_dict)
    # 3. load the new state dict
    self.load_state_dict(model_dict)


In [22]:
def load_weights(model, pretrained = r'..\Models\Tiramisu\tiramisu103.pth', n_classes= 2):
    pretrained_dict = torch.load(pretrained)
    model_dict = model.state_dict()

    # 1. filter out unnecessary keys
    #pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    tmp_pretrained_dict = {}
    for k, v in list(pretrained_dict.items())[:-2]:
      
      if k in model_dict:
        print(f'match {k}')
        tmp_pretrained_dict.update({k: v})
      else:
        print(f'unmatch {k}')
    # 2. overwrite entries in the existing state dict
    model_dict.update(tmp_pretrained_dict)

    # 3. load the new state dict
    model.load_state_dict(model_dict)

    return model


In [24]:
model_one_2 = load_weights(model = model_one)


match module.model.0.weight
match module.model.0.bias
match module.model.1.dense_block.module.0.module.BN.weight
match module.model.1.dense_block.module.0.module.BN.bias
match module.model.1.dense_block.module.0.module.BN.running_mean
match module.model.1.dense_block.module.0.module.BN.running_var
match module.model.1.dense_block.module.0.module.BN.num_batches_tracked
match module.model.1.dense_block.module.0.module.CONV.weight
match module.model.1.dense_block.module.0.module.CONV.bias
match module.model.1.dense_block.module.1.module.BN.weight
match module.model.1.dense_block.module.1.module.BN.bias
match module.model.1.dense_block.module.1.module.BN.running_mean
match module.model.1.dense_block.module.1.module.BN.running_var
match module.model.1.dense_block.module.1.module.BN.num_batches_tracked
match module.model.1.dense_block.module.1.module.CONV.weight
match module.model.1.dense_block.module.1.module.CONV.bias
match module.model.1.dense_block.module.2.module.BN.weight
match module.

In [25]:
x = model.module.model[5].dense_block.module[0].module[-2].weight
y = model_one.module.model[5].dense_block.module[0].module[-2].weight
print(x.ne(y).sum())

tensor(0, device='cuda:0')


In [19]:
model.module.LastLayer

Sequential(
  (point_wise): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
  (softmax): LogSoftmax()
)

In [21]:
model_one_2.module.LastLayer

Sequential(
  (point_wise): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
  (softmax): LogSoftmax()
)

In [17]:
torch.save(model_one_2.state_dict(), path)

In [ ]:
# Mobile Net Weight transfer

In [1]:
import sys
sys.path.append('../../Backbone/MobileNetv2')
from MobileNetv2 import *

In [10]:
# my model
model = MobileNetv2(n_classes = 1000)

In [11]:
# torch hub model
torchModel = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)

Using cache found in C:\Users\admin/.cache\torch\hub\pytorch_vision_v0.6.0


In [12]:
for (k, v), (tk, tv) in zip(list(model.state_dict().items()), torchModel.state_dict().items()) :
    print(k, ":", v.size()," <=> ", tk, ":", tv.size())

model.0.FirstBlock_Conv2D.weight : torch.Size([32, 3, 3, 3])  <=>  features.0.0.weight : torch.Size([32, 3, 3, 3])
model.0.FirstBlock_BN.weight : torch.Size([32])  <=>  features.0.1.weight : torch.Size([32])
model.0.FirstBlock_BN.bias : torch.Size([32])  <=>  features.0.1.bias : torch.Size([32])
model.0.FirstBlock_BN.running_mean : torch.Size([32])  <=>  features.0.1.running_mean : torch.Size([32])
model.0.FirstBlock_BN.running_var : torch.Size([32])  <=>  features.0.1.running_var : torch.Size([32])
model.0.FirstBlock_BN.num_batches_tracked : torch.Size([])  <=>  features.0.1.num_batches_tracked : torch.Size([])
model.1.module.EXPANDED_CONV_DW_CONV_0.weight : torch.Size([32, 1, 3, 3])  <=>  features.1.conv.0.0.weight : torch.Size([32, 1, 3, 3])
model.1.module.EXPANDED_CONV_DW_BN_0.weight : torch.Size([32])  <=>  features.1.conv.0.1.weight : torch.Size([32])
model.1.module.EXPANDED_CONV_DW_BN_0.bias : torch.Size([32])  <=>  features.1.conv.0.1.bias : torch.Size([32])
model.1.module.EXPA

In [13]:
pretrained_dict = {}
for (k, v), (tk, tv) in zip(list(model.state_dict().items()), torchModel.state_dict().items()) :
    #print(k, ":", v.size()," <=> ", tk, ":", tv.size())
    pretrained_dict.update({k : tv})
update_state_dict = model.state_dict()
update_state_dict.update(pretrained_dict)
model.load_state_dict(update_state_dict)

<All keys matched successfully>

In [14]:
# check weight
for (k, v), (tk, tv) in zip(list(model.state_dict().items()), torchModel.state_dict().items()) :
    print(k, " <=> ", tk, ":", v.ne(tv).sum()) # hpw many differences => 0 : correct
    

model.0.FirstBlock_Conv2D.weight  <=>  features.0.0.weight : tensor(0)
model.0.FirstBlock_BN.weight  <=>  features.0.1.weight : tensor(0)
model.0.FirstBlock_BN.bias  <=>  features.0.1.bias : tensor(0)
model.0.FirstBlock_BN.running_mean  <=>  features.0.1.running_mean : tensor(0)
model.0.FirstBlock_BN.running_var  <=>  features.0.1.running_var : tensor(0)
model.0.FirstBlock_BN.num_batches_tracked  <=>  features.0.1.num_batches_tracked : tensor(0)
model.1.module.EXPANDED_CONV_DW_CONV_0.weight  <=>  features.1.conv.0.0.weight : tensor(0)
model.1.module.EXPANDED_CONV_DW_BN_0.weight  <=>  features.1.conv.0.1.weight : tensor(0)
model.1.module.EXPANDED_CONV_DW_BN_0.bias  <=>  features.1.conv.0.1.bias : tensor(0)
model.1.module.EXPANDED_CONV_DW_BN_0.running_mean  <=>  features.1.conv.0.1.running_mean : tensor(0)
model.1.module.EXPANDED_CONV_DW_BN_0.running_var  <=>  features.1.conv.0.1.running_var : tensor(0)
model.1.module.EXPANDED_CONV_DW_BN_0.num_batches_tracked  <=>  features.1.conv.0.1.nu

In [15]:
torch.save(model.state_dict(), '../../Models/MobileNetv2/mnv2.pth')

In [16]:
import torch.nn as nn 
do = nn.Dropout(0.4)


In [18]:
help(do)

Help on Dropout in module torch.nn.modules.dropout object:

class Dropout(_DropoutNd)
 |  Dropout(p=0.5, inplace=False)
 |  
 |  During training, randomly zeroes some of the elements of the input
 |  tensor with probability :attr:`p` using samples from a Bernoulli
 |  distribution. Each channel will be zeroed out independently on every forward
 |  call.
 |  
 |  This has proven to be an effective technique for regularization and
 |  preventing the co-adaptation of neurons as described in the paper
 |  `Improving neural networks by preventing co-adaptation of feature
 |  detectors`_ .
 |  
 |  Furthermore, the outputs are scaled by a factor of :math:`\frac{1}{1-p}` during
 |  training. This means that during evaluation the module simply computes an
 |  identity function.
 |  
 |  Args:
 |      p: probability of an element to be zeroed. Default: 0.5
 |      inplace: If set to ``True``, will do this operation in-place. Default: ``False``
 |  
 |  Shape:
 |      - Input: :math:`(*)`. Input